# Step 3: Cap Haitien GTFS Accessibility Analysis
Running travel time to closest facility for health facilities, schools, and markets

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np
from shapely.geometry import Point

In [3]:
import os, sys

In [4]:
# Get reference to GOSTNets
import sys
sys.path.append(r'C:\repos\GOSTnets')
import GOSTnets as gn

### Load graphs

In [5]:
# read back your graphs from step 2 from you saved pickle
G_service0001 = nx.read_gpickle(r"temp\gtfs_export_cap_haitien_merged_impute_walk_v5_service0001.pickle")
G_service0002 = nx.read_gpickle(r"temp\gtfs_export_cap_haitien_merged_impute_walk_v5_service0002.pickle")

In [6]:
#print(nx.info(G))

In [7]:
#list(G.edges)[0:15]

In [8]:
graphs = {'G_service0001': G_service0001, 'G_service0002': G_service0002}

In [9]:
# print info about the largest sub-graph
#print(nx.info(G_largest))

### Load Data

In [10]:
# load origins
origins = gpd.read_file(r"input_folder\cap_haitien_worldpop_pts2.shp")

In [11]:
# load destinations
health_destinations = gpd.read_file(r"input_folder\cap_haitien_health_pts.shp")
school_destinations = gpd.read_file(r"input_folder\cap_haitien_schools_DPCE_20180709.shp")
market_destinations = gpd.read_file(r"input_folder\Marches_Haiti_IICA1977.shp")

In [12]:
destinations_dict = {'health_destinations': health_destinations, 'school_destinations': school_destinations, 'market_destinations': market_destinations}

#### For each destination for each type of graph do an accessibility analysis

In [13]:
for G in graphs.items():
    # Origins remain the same for each type of accessibility
    # insert origins
    origins = gpd.read_file(r"input_folder\cap_haitien_worldpop_pts2.shp")
    # snap the origins to the road graph
    snapped_origins = gn.pandana_snap(G[1], origins, source_crs = 'epsg:4326', target_crs = 'epsg:32619')
    # filter out origins with a distance more than 2km
    snapped_origins_filtered = snapped_origins[snapped_origins.NN_dist < 2000]
    originNodes = list(snapped_origins_filtered['NN'].unique())
    for destination in destinations_dict.items():
        # snap the destinations to the road graph
        snapped_destinations = gn.pandana_snap(G[1], destination[1], source_crs = 'epsg:4326', target_crs = 'epsg:32619')
        destinationsNodes = list(snapped_destinations['NN'].unique())
        # Calculate OD Matrix
        OD_matrix = gn.calculate_OD(G[1], originNodes, destinationsNodes, fail_value=-1, weight='length')
        avg_trip_time = np.mean(OD_matrix)
        print(f"average trip time for {destination[0]} with the {G[0]} graph is: {avg_trip_time/60} minutes")
        # calculate accessibility
        # For each row, the closest facility is the smallest value in the row
        closest_facility_per_origin = OD_matrix.min(axis=1)
        results = pd.DataFrame([originNodes, closest_facility_per_origin]).transpose()
        colName = "travel_time_to_closest_facility"
        results.columns = ['NN', colName]
        output = snapped_origins_filtered.copy()
        output = pd.merge(output, results, on="NN")
        #convert travel_time_to_closest_facility to number
        output["travel_time_to_closest_facility"] = pd.to_numeric(output["travel_time_to_closest_facility"])
        # save a shapefile...
        destinations_gpd = gpd.GeoDataFrame(output, crs = "epsg:4326", geometry = 'geometry')
        destinations_gpd.to_file(fr"output_folder\cap_haitien_accessibility_{destination[0]}_{G[0]}.shp")


average trip time for health_destinations with the G_service0001 graph is: 175.53994741170698 minutes
average trip time for school_destinations with the G_service0001 graph is: 163.54374861129423 minutes
average trip time for market_destinations with the G_service0001 graph is: 237.239912326493 minutes
average trip time for health_destinations with the G_service0002 graph is: 174.96226560365415 minutes
average trip time for school_destinations with the G_service0002 graph is: 162.99757422166755 minutes
average trip time for market_destinations with the G_service0002 graph is: 236.83141094660292 minutes
